In [1]:
from __future__ import division
%matplotlib inline
import CameraNetwork
import CameraNetwork.global_settings as gs
import cPickle
import cv2
import datetime
import Image
import ImageDraw
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import time
import tqdm
from tqdm import tqdm_notebook
import traceback
plt.rcParams['figure.figsize'] = (8.0, 8.0)
idx = pd.IndexSlice

i:\amit\office backup\amit\studies\cameranetwork_git\CameraNetwork\transformation_matrices.py:1899: UserWarning: failed to import module _transformations
  warnings.warn("failed to import module %s" % name)


In [9]:
c = CameraNetwork.CLIclient()
proxy_params = CameraNetwork.retrieve_proxy_parameters()
c.start(proxy_params)
time.sleep(3)
print c.client_instance.servers

ERROR:root:Failed retrieving proxy parameters:
Traceback (most recent call last):
  File "i:\amit\office backup\amit\studies\cameranetwork_git\CameraNetwork\internet.py", line 56, in retrieve_proxy_parameters
    response = urllib2.urlopen(req)
  File "C:\Anaconda2\lib\urllib2.py", line 154, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Anaconda2\lib\urllib2.py", line 435, in open
    response = meth(req, response)
  File "C:\Anaconda2\lib\urllib2.py", line 548, in http_response
    'http', request, response, code, msg, hdrs)
  File "C:\Anaconda2\lib\urllib2.py", line 473, in error
    return self._call_chain(*args)
  File "C:\Anaconda2\lib\urllib2.py", line 407, in _call_chain
    result = func(*args)
  File "C:\Anaconda2\lib\urllib2.py", line 556, in http_error_default
    raise HTTPError(req.get_full_url(), code, msg, hdrs, fp)
HTTPError: HTTP Error 404: Not Found



['109', '109L', '110', '110L', '111', '111L', '112L', '113', '113L', '114L', '115', '115L', '116', '116L', '117', '118', '120']


In [3]:
dfs = {}
tnb1 = tqdm_notebook(c.client_instance.servers, leave=False)
for server in tnb1:
    tnb1.set_description("Server {}".format(server))
    tnb1.refresh()
    days_list = c.days(server)[1][0]["days_list"]
    tmp_dfs = []
    tnb2 = tqdm_notebook(days_list, leave=False)
    for day in tnb2:
        try:
            tnb2.set_description(day)
            tnb2.refresh()
            tmp_dfs.append(c.query(server_id=server, query_day=datetime.datetime.strptime(day, "%Y_%m_%d").date())[0])
        except:
            tqdm.tqdm.write("Failed loading day {} of server {}".format(day, server))
    dfs[server] = pd.concat(tmp_dfs, axis=0)

In [7]:
del dfs["115"]["longitue"]

In [8]:
with open("thumbnails.pkl", "wb") as f:
    cPickle.dump(dfs, f)

## Analyze the thumbnails database

In [2]:
with open("thumbnails.pkl", "rb") as f:
    dfs = cPickle.load(f)

In [3]:
dfs["109"].head()

path  \
Time                hdr                                                      
2017-01-25 12:27:00 0    /home/odroid/captured_images/2017_01_25/148534...   
                    1    /home/odroid/captured_images/2017_01_25/148534...   
                    2    /home/odroid/captured_images/2017_01_25/148534...   
                    3    /home/odroid/captured_images/2017_01_25/148534...   
2017-01-25 12:28:00 0    /home/odroid/captured_images/2017_01_25/148534...   

                         longitude   latitude  altitude  serial_num  
Time                hdr                                              
2017-01-25 12:27:00 0    35.024963  32.775776     229.0  4102820380  
                    1    35.024963  32.775776     229.0  4102820380  
                    2    35.024963  32.775776     229.0  4102820380  
                    3    35.024963  32.775776     229.0  4102820380  
2017-01-25 12:28:00 0    35.024963  32.775776     229.0  4102820380

In [4]:
def custom_resampler(array_like):
    if array_like.shape[0] == 0:
        return None
    return array_like.iloc[-1]

In [5]:
df = dfs["109"]
df_thumbs = df.loc[idx[:, "2"], idx[:]].resample("30T", level=0, label='right').apply(custom_resampler).dropna()

In [6]:
df_thumbs.head()

,path,longitude,latitude,altitude,serial_num
Time,,,,,
2017-01-25 12:30:00,/home/odroid/captured_images/2017_01_25/148534...,35.024963,32.775776,229.0,4102820380
2017-01-25 13:00:00,/home/odroid/captured_images/2017_01_25/148534...,35.024963,32.775776,229.0,4102820380
2017-01-25 13:30:00,/home/odroid/captured_images/2017_01_25/148535...,35.024963,32.775776,229.0,4102820380
2017-01-25 14:00:00,/home/odroid/captured_images/2017_01_25/148535...,35.024963,32.775776,229.0,4102820380
2017-01-25 14:30:00,/home/odroid/captured_images/2017_01_25/148535...,35.024963,32.775776,229.0,4102820380


## Download thumbnails

In [ ]:
JPEG_QUALITY = 95
tn_dfs = {}
servers_list = sorted(dfs.keys())
for server in tqdm_notebook(servers_list):
    tqdm.tqdm.write("Processing server: {}".format(server))
    df = dfs[server]
    df_thumbs = df.loc[idx[:, "2"], idx[:]].resample("30T", level=0, label='right').apply(custom_resampler).dropna()
    
    base_path = os.path.join("thumbnails", server)
    if not os.path.exists(base_path):
        os.makedirs(base_path)

    tn_paths = []
    for ind, row in tqdm_notebook(df_thumbs.iterrows(), total=df_thumbs.shape[0], leave=False):
        try:
            array_path = row["path"]
            array_path = os.path.splitext(array_path)[0]

            #
            # Parse the time and exposure
            #
            tmp_parts = os.path.split(array_path)[-1].split('_')
            time_ind = datetime.datetime(*[int(i) for i in tmp_parts[1:-1]])

            img = c.seek(server, time_ind, "2", JPEG_QUALITY, 201)[0][0]
            img = Image.fromarray(img)
            tn_path = os.path.join(base_path, "{}.jpg".format(ind.strftime("%Y_%m_%d_%H_%M_%S")))
            img.save(tn_path)
            tn_paths.append(tn_path)
        except Exception, e:
            traceback.print_exc()
            tn_paths.append(None)
            raise

    df_thumbs["thumbnail"] = tn_paths
    
    tn_dfs[server] = df_thumbs

Processing server: 109


In [ ]:
with open("thumbnails_downloaded.pkl", "wb") as f:
    cPickle.dump(tn_dfs, f)